# Subtitle generator

- Uses [OpenAI's Whisper](https://github.com/openai/whisper) for generating subtitles for media files using AI

## Creating user input UI

- URLs for videos go in the textbox below
- One URL per line

In [ ]:
import ipywidgets as wd
from enum import Enum

class Models(Enum):
  TURBO = "turbo"
  LARGE = "large"
  MEDIUM = "medium"
  SMALL = "small"
  BASE = "base"
  TINY = "tiny"

class Tasks(Enum):
  TRANSCRIBE = "transcribe"
  TRANSLATE = "translate"

dd_model = wd.Dropdown(
    description="Model: ",
    options = [model.value for model in Models],
    index = 0,
    )
rb_task = wd.RadioButtons(
    description="Task: ",
    options = [task.value for task in Tasks],
    index = 0,
    disabled = True,
    )
cb_english = wd.Checkbox(
    description="English-only model?",
    value = False,
    disabled = True,
    )
ta_urls = wd.Textarea(
    placeholder="Type URLs for videos",
    description="URLs",
  )
cb_follow_playlist = wd.Checkbox(
    description="Follow playlist?",
    value = False,
    )

def check_valid_model(selected_model):
  model = selected_model.new
  if  (model == Models.TURBO.value or model == Models.LARGE.value):
    cb_english.disabled = True
  else:
    cb_english.disabled = False

def check_valid_task(selected_model):
  model = selected_model.new

  # Turbo model can't translate, so force transcription
  if  (model == Models.TURBO.value):
    rb_task.disabled = True
    rb_task.value = Tasks.TRANSCRIBE.value
  else:
    rb_task.disabled = False

def validate_fields(selected_model):
  check_valid_model(selected_model)
  check_valid_task(selected_model)

dd_model.observe(validate_fields, names="value")

audio_folder="audio"
subtitles_folder="subtitles"
zipfile="subtitles.zip"

display(
  ta_urls,
  dd_model,
  rb_task,
  cb_english,
  cb_follow_playlist
  )

## Setting up the environment

In [ ]:
%%capture
!pip install -U openai
!pip install -U cohere
!pip install -U openai-whisper

In [ ]:
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install ffmpeg-python
!sudo apt install ffmpeg

In [ ]:
%cd /content
!rm -rfd {audio_folder}
!rm -rfd {subtitles_folder}
!mkdir -pv {audio_folder}

## Retrieving videos

If no URLs are passed to the textbox on the UI above, an upload prompt is show to get videos from the user's machine

In [ ]:
urls = ta_urls.value.splitlines()

if len(urls) == 0:
  import shutil
  import os

  # upload videos
  uploaded = files.upload()
  for filename in uploaded.keys():
    dst_path = os.path.join(audio_folder, filename)
    print(f'move {filename} to {dst_path}')
    shutil.move(filename, dst_path)
else:
  for url in urls:
    %cd {audio_folder}
    !yt-dlp {("--yes-playlist" if cb_follow_playlist.value == True else "")} -x {url}

%cd /content

# Sanity check
if len(os.listdir(audio_folder)) == 0:
  except "No files in audio folder"

## Generating subtitles

In [ ]:
# Done like this so linter stops complaining
model = f"{dd_model.value}" + (".en" if cb_english.value else "")
task = rb_task.value

args = [
  "--verbose False",
  f"--model {model}",
  f"--task {task}",
  f"--output_dir {subtitles_folder}",
  f"{audio_folder}/*"
]

args = " ".join(args)
!whisper {args}

## Downloading results

In [ ]:
!rm -f {zipfile}
!zip -r {zipfile} {subtitles_folder}

In [ ]:
from google.colab import files
files.download(zipfile)